<a href="https://colab.research.google.com/github/ctruciosm/ACA124/blob/main/Intro_Simplex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução os Métodos Quantitativos em Administração com o Software R
## By Prof. Carlos Trucíos

## 1. Instalando R

### 1.1 Instalando R no computador

- Instalar R [Windows](https://cran.r-project.org/bin/windows/base/), [Outros SO](https://cran.r-project.org/).
- Usuários Windows devem também instalar Rtools [Link](https://cran.r-project.org/bin/windows/Rtools/)
- Instalar Rstudio [Link](https://www.rstudio.com/products/rstudio/download/) (escolher RStudio Desktop Free)
- Uma vez que tenha tudo instalado, recomendo assistir o seguinte [video](https://www.youtube.com/watch?v=5SMJsKamuSQ&list=PLrqJCLOY0EkDc3zIhA0-Lk-M12FnReWwa&index=3&t=3130s) onde são apresentados os primeiros passos com o R.

### 1.2 R na nuvem

- Rstudio cloud (precisa criar uma conta) [Link](https://rstudio.cloud/)
- Google Colab [https://colab.to/r](https://colab.to/r) (precisa ter conta gmail)


# 2. O Método Simplex

Várias funções básicas vem já disponíveis no _pacote base_ do **R**. Contudo, para tarefas específicas precisaremos instalar pacotes específicos.

> Um pacote no R é um módulo com funções e/ou _datasets_ desenhado para fazer tarefas específicas. Os pacotes são desenvolvidos por usuários da comunidade R no mundo para que possam ser utilizados por outros usuários da comunidade R.



In [1]:
# Instalando os pacotes boot e lpSolve
install.packages("boot")     # A estrutura é sempre install.packages("nome_do_pacote_entre_aspas")
install.packages("lpSolve") 

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



A instalação do pacote é feita apenas uma única vez no computador, mas cada vez que formos utilizar o pacote precisamos chamá-los (carregá-los).

In [2]:
# Carregando pacotes
library(boot)
library(lpSolve)

Veremos duas opções para trabalhar com o método simplex no R:

- A função `simplex()` do pacote `boot`.
- A função `lp()` do pacote `lpSolve`.


**Importante:** antes de entender como utilizar estas funções, precisamos primeiro escrever o modelo matemático na sua forma matricial.



Seja o problema de programação linear com $n$ variáveis de decisão:

$$\text{Max: } Z = c_1 X_1 + c_2 X_2 + \cdots, c_n X_n,$$

sujeito às restrições:

\begin{align}
    & a_{11} X_1 + a_{12} X_2 + \cdots + a_{1n} X_n \leq b_1 \\
    & a_{21} X_1 + a_{22} X_2 + \cdots + a_{2n} X_n \leq b_2 \\
    & \vdots \quad \quad \quad \quad \quad    \cdots  \quad   \quad \quad  \quad \quad \vdots \\
    & a_{m1} X_1 + a_{m2} X_2 + \cdots + a_{mn} X_n \leq b_m \\
\end{align}

e $X_i \geq 0 \quad \forall i = 1, \cdots, n.$

Se fizermos: 
$$ X = \left[ {\begin{array}{cc}
    X_1 \\
    X_2 \\
    \vdots \\
    X_n \\
  \end{array} } \right] \quad, \quad A =
  \left[ {\begin{array}{cc}
    a_{11} & a_{12}  & \cdots & a_{1n} \\
    a_{21} & a_{22}  & \cdots & a_{2n} \\
    \vdots &         & \cdots  & \vdots \\
    a_{m1} & a_{m2}  & \cdots & a_{mn} \\
  \end{array} } \right] \quad \text{ e } \quad b = \left[ {\begin{array}{cc}
    b_{1} \\
    b_{2} \\
    \vdots \\
    b_{m} \\
  \end{array} } \right],$$
  em que $X$ é uma matriz de dimensão $n \times 1$, $A$ é uma matriz de dimensão $m \times n$ e $b$ é uma matriz de dimensão $m \times 1$. Então,  as restrições funcionais na forma matricial seriam:

  $$A X \leq b$$

  > Repare que como a matriz $A$ tem dimensão $m \times n$ e a matrix $X$ tem dimensão $n \times 1$, a multiplicação de matrices faz sentido e o resultado será uma matriz de dimensão $m \times 1$ (que coincide com a dimensão da matriz $b$).



### Exemplo:

Consideremos o seguinte problema (Caso 0: vidros UFRJ). 

$$\text{Max: } Z = 3 X_1 + 5 X_2,$$

sujeito às restrições

\begin{align}
    & X_1 & \leq 4 \\
    &  X_2 & \leq 6 \\
    & 3X_1 + 2X_2 & \leq 18 \\
\end{align}

e $X_1, X_2 \geq 0.$

Na forma matricial temos:


$$ X= \left[ {\begin{array}{cc}
    X_1  \\
    X_2  \\
  \end{array} } \right] \text{,} \quad A =
  \left[ {\begin{array}{cc}
    1 & 0  \\
    0 & 1 \\
    3 & 2 \\
  \end{array} } \right] \quad \text{ e } \quad b = \left[ {\begin{array}{cc}
    4  \\
    6  \\
    18 \\
  \end{array} } \right].$$

  Assim, a restrição funcional é simplesmente $AX \leq b.$

In [3]:
C = c(3, 5)  # vetor com os coeficientes da função objetivo
A = matrix(c(1, 0, 3, 0, 1, 2), ncol = 2)   # preencher coluna a coluna
b = c(4, 6 , 18) # vetor com os 'lados direitos' das restrições funcionais

**Observação:** a letra `c` em minúscula é utilizado no R para definir vetores, por isso estou utilizando `C` em maiúscula para assim evitar confussão.

In [4]:
C    # Para 'printar' o que está no objeto criado C

[1] 3 5

In [5]:
A   # Para 'printar' o que está no objeto criado A

1,0
0,1
3,2


In [6]:
b   # Para 'printar' o que está no objeto criado b

[1]  4  6 18

#### Método 1: função simplex() do pacote boot

In [7]:
simplex_metodo_1 = simplex(C, A, b, maxi = TRUE)

In [8]:
# Para visualisar o resultado inteiro que foi salvo em simplex_metodo_1
simplex_metodo_1


Linear Programming Results

Call : simplex(a = C, A1 = A, b1 = b, maxi = TRUE)

Maximization Problem with Objective Function Coefficients
x1 x2 
 3  5 


Optimal solution has the following values
x1 x2 
 2  6 
The optimal value of the objective  function is 36.

In [9]:
# Para visualisar apenas a solução ótima
simplex_metodo_1$soln

x1 x2 
 2  6

> A função `simplex()` otimiza a função $Z = c_1 X_1 + c_2 X_2 + \cdots c_n X_n$ sujeito às restrições:

- $AX \leq b,$
- $X \geq 0$

Tanto maximização (`maxi = TRUE`) como minimização (`maxi = FALSE`) são permitidos.

#### Método 2: função lp() do pacote lpSolve

In [10]:
simplex_metodo_2 = lp("max", C, A, rep("<=",3), b)

In [11]:
simplex_metodo_2$solution

[1] 2 6

**Observação:** o quarto argumento na função `lp()`permite especificar explicatamente a direção da restrição, sendo permitido opções como `=, >, <, >=, <=`.

Repare que cada função tem uma forma diferente de incluir os dados e fornecer os resultados. Quando utilizamos uma função com frequência já estamos acostumados em como introduzir os dados na função, mas muitas vezes isto é completamente desconhecido para nós. Nesses casos, basta "perguntar ao R" como utilizar a função através do comando `help(nome_da_função)` ou `?nome_da_função`.